In [1]:
import warnings
warnings.filterwarnings('ignore')
import yaml
import pandas as pd

In [2]:
from multi_agents import MultiAgentSystem

In [3]:
with open("/Users/sigmoid/Downloads/llm_keys.yaml", "r") as f:
    llm_keys = yaml.safe_load(f)

In [4]:
file_path = '/Users/sigmoid/Documents/PepsiCo - Gen AI PoC/lift_bot/src'

In [5]:
# Load data (MODIFIED for new structure)
df_expenses = pd.read_csv(f"{file_path}/data/Expense.csv")

In [6]:
# Convert to correct format
df_expenses["Expense"] = (
    df_expenses["Expense"]
    .str.replace(",", "", regex=False)
    .astype(float)
    .astype(int)
)

In [7]:
df_expenses.head()

,Country,Year,Month,Category,Brand,Data Type,Tier 1,Tier 2,Tier 3,Expense,Status,Count
0,Brazil,2022,1,CSD Colas,PEPSI,A&M,PULL - Non working,Agency Fees,Creative Agency Fees,125,Approved,1
1,Brazil,2022,1,CSD Colas,PEPSI,A&M,PULL - Non working,Agency Fees,Media Agency Fees,20603,Approved,1
2,Brazil,2022,1,CSD Colas,PEPSI,A&M,PULL - Non working,Agency Fees,Social Media/Influence Marketing,847,Approved,1
3,Brazil,2022,1,CSD Colas,PEPSI,A&M,PULL - Non working,Insight,Advanced Analytics,39123,Approved,1
4,Brazil,2022,1,CSD Colas,PEPSI,A&M,PULL - Non working,Insight,Consumer Tracking,1951,Approved,1


In [8]:
df_budget = pd.read_csv(f"{file_path}/data/Budget_RB.csv")

In [9]:
df_budget.head()

,Region,Country,Year,Brand,Tier 1,Tier 2,Tier 3,Pep (%),Budget
0,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Ad Production,Digital Ad Production,0.0,0
1,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Ad Production,Other Ad Production,0.0,0
2,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Ad Production,TV Print Radio OOH Prod,100.0,491755
3,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Agency Fees,Creative Agency Fees,0.0,0
4,LAB Mexico,Mexico,2025,Be-Light,Pull-Non-Working,Agency Fees,In Store and POS Design/Development,0.0,0


In [10]:
multi_agent_obj = MultiAgentSystem(
    model_name='gpt-4o',
    api_key=llm_keys['open_ai'],
    expense_dataset=df_expenses,
    budget_dataset=df_budget,
    plot_path='jupyter_plots',
)

In [11]:
# Expense
# question = 'What are the Tier 1 expenses in 2024 for Brazil?'
# question = "How are the Tier 1 expenses in CY varying for Mexico & Brazil for Pepsi brand?"

In [12]:
# Budget
question = 'What is the total budget allocated for Mexico in 2025?'
# question = "Can you give me split of Tier 1 items for each Brazil & Mexico for 2025?"

In [13]:
result = multi_agent_obj.graph.nodes['supervisor'].invoke(
    {
        'question': question
    }
)

In [14]:
result

{'result': {'thought_process': 'The user is asking for a specific data analysis regarding the budget allocated for Mexico in 2025. This requires accessing and analyzing budget data, which falls under the responsibilities of the Insight Agent.',
  'next': 'Insight Agent',
  'enriched_question': 'What is the total budget allocated for Mexico in 2025?'},
 'messages': [AIMessage(content='The user is asking for a specific data analysis regarding the budget allocated for Mexico in 2025. This requires accessing and analyzing budget data, which falls under the responsibilities of the Insight Agent.', additional_kwargs={}, response_metadata={}, name='supervisor')],
 'next': 'Insight Agent'}

In [15]:
result2 = multi_agent_obj.graph.nodes['Insight Agent'].invoke(
    {
        'enriched_question': result['result']['enriched_question']
    }
)



> Entering new AgentExecutor chain...

Invoking: `analyze_budget_data` with `Calculate the total budget allocated for Mexico in 2025.`


{'approach': 'To calculate the total budget allocated for Mexico in 2025, I will:\n1. Filter the dataset for entries where the "Country" is "Mexico" and the "Year" is 2025.\n2. Sum the "Budget" column for these filtered entries to get the total budget.', 'answer': 'The total budget allocated for Mexico in 2025 is 5742565224.', 'figure': None, 'code': "import pandas as pd\n\n# Filter the dataset for Mexico in 2025\nfiltered_df = df[(df['Country'].str.strip().str.upper() == 'MEXICO') & (df['Year'] == 2025)]\n\n# Calculate the total budget\ntotal_budget_mexico_2025 = filtered_df['Budget'].sum()\ntotal_budget_mexico_2025", 'chart_code': None}<answer>The total budget allocated for Mexico in 2025 is $5,742,565,224.</answer><graph>None</graph>

> Finished chain.


In [16]:
result2.keys()

dict_keys(['input', 'history', 'output', 'recorder_steps', 'agent'])

In [17]:
result2['output']

'<answer>The total budget allocated for Mexico in 2025 is $5,742,565,224.</answer><graph>None</graph>'

In [18]:
len(result2['recorder_steps'])

2

In [19]:
len(result2['recorder_steps'][-2:])

2

In [20]:
result2['input']

'What is the total budget allocated for Mexico in 2025?'

In [21]:
result2['recorder_steps'][0]['observation']['approach']

'To calculate the total budget allocated for Mexico in 2025, I will:\n1. Filter the dataset for entries where the "Country" is "Mexico" and the "Year" is 2025.\n2. Sum the "Budget" column for these filtered entries to get the total budget.'

In [22]:
result2['recorder_steps'][0]['observation']['answer']

'The total budget allocated for Mexico in 2025 is 5742565224.'

In [23]:
result2['recorder_steps'][1]

{'final_answer': '<answer>The total budget allocated for Mexico in 2025 is $5,742,565,224.</answer><graph>None</graph>'}

In [24]:
result2['output'] == result2['recorder_steps'][1]['final_answer']

True

In [25]:
question

'What is the total budget allocated for Mexico in 2025?'

In [26]:
result3 = multi_agent_obj.graph.invoke(
    {
        'question': question
    }
)



> Entering new AgentExecutor chain...

Invoking: `analyze_budget_data` with `Find the total budget allocated for Mexico in 2025.`


{'approach': 'To find the total budget allocated for Mexico in 2025, I will:\n1. Filter the dataset for entries where the "Country" is "Mexico" and the "Year" is 2025.\n2. Sum the "Budget" column for these filtered entries to get the total budget.', 'answer': 'The total budget allocated for Mexico in 2025 is 5742565224.', 'figure': None, 'code': "import pandas as pd\n\n# Filter the dataset for Mexico in 2025\nfiltered_df = df[(df['Country'].str.strip().str.upper() == 'MEXICO') & (df['Year'] == 2025)]\n\n# Calculate the total budget\ntotal_budget_mexico_2025 = filtered_df['Budget'].sum()\ntotal_budget_mexico_2025", 'chart_code': None}<answer>The total budget allocated for Mexico in 2025 is $5,742,565,224.</answer><graph>None</graph>

> Finished chain.


In [27]:
result3

{'result': {'thought_process': 'The user asked for the total budget allocated for Mexico in 2025, which required data analysis. The Insight Agent provided the answer, stating that the total budget is $5,742,565,224. Since the question has been answered, no further action is needed.',
  'next': 'FINISH',
  'direct_response': 'The total budget allocated for Mexico in 2025 is $5,742,565,224.',
  'enriched_question': 'What is the total budget allocated for Mexico in 2025?'},
 'messages': [AIMessage(content='The user asked for the total budget allocated for Mexico in 2025, which required data analysis. The Insight Agent provided the answer, stating that the total budget is $5,742,565,224. Since the question has been answered, no further action is needed.', additional_kwargs={}, response_metadata={}, name='supervisor')],
 'next': 'FINISH'}